#Descrição do Trabalho

Contexto
 
O departamento de crédito ao consumidor de um banco deseja automatizar o processo de tomada de decisão para aprovação das linhas de crédito Home Equity (Crédito com Garantia em Imóvel) do patrimônio líquido. Para fazer isso, eles seguirão as recomendações da Lei da Igualdade de Oportunidades de Crédito para criar um modelo de pontuação de crédito derivado empiricamente e estatisticamente sólido. O modelo será baseado em dados coletados de solicitantes recentes concedidos crédito através do processo atual de concessão de empréstimos. O modelo será construído a partir de ferramentas de modelagem preditiva, mas o modelo criado deve ser suficientemente interpretável para fornecer um motivo para qualquer ação adversa (rejeição).

Conteúdo

O conjunto de dados de Home Equity (HMEQ) (Crédito com Garantia em Imóvel) contém informações de linha de base e de desempenho de empréstimos para 5.960 empréstimos recentes de home equity. O alvo (BAD) é uma variável binária que indica se um requerente acabou de tornar inadimplente ou é um inadimplente conntumaz. Esse desfecho adverso ocorreu em 1.189 casos (20%). Para cada candidato, foram registradas 12 variáveis de entrada.


Motivação

E se você puder prever clientes que não pagam seus empréstimos.

BAD

1 = cliente inadimplente no empréstimo 0 = empréstimo recebido

LOAN

Montante do pedido de empréstimo

MORTDUE

Valor devido da hipoteca existente

VALUE

Valor da propriedade atual

REASON

DebtCon = consolidação da dívida HomeImp = melhoria da casa

JOB

Seis categorias ocupacionais

YOJ

Anos no emprego atual

DEROG

Número de principais relatórios depreciativos

DELINQ

Número de linhas de crédito inadimplentes

CLAGE

Idade da linha comercial mais antiga em meses

NINQ

Número de linhas de crédito recentes

CLNO

Número de linhas de crédito

DEBTINC

Razão dívida / rendimento


JOB - Valores dos Campos

Mgr-Manager -> trabalho de gerente
Office -> trabalho de escritório
Other -> outros trabalhos
ProfExe -> trabalho profissional e/ou executivo
Sales -> trabalho com vendas
Self -> trabalho por conta própria

In [ ]:
# Importando as Bibliotecas
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import itertools

from sklearn.metrics import balanced_accuracy_score
from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.metrics import geometric_mean_score



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# Importando os dados para o dataframe
dfori = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

In [ ]:
# Exibindo quantidade de linhas e colunas
print('dfori:', dfori.shape)

In [ ]:
# Exibindo colunas e seus tipos de dados
dfori.info()

In [ ]:
# Mostrando alguns exenplos da base de dados
dfori.sample(5).T

In [ ]:
# Analisando os dados da coluna BAD
dfori['BAD'].value_counts()

In [ ]:
# Exibindo gráfico de barras da coluna BAD
dfori['BAD'].value_counts().plot.bar()

In [ ]:
# Analisando os dados da coluna JOB
dfori['JOB'].value_counts()

In [ ]:
# Exibindo gráfico de barras da coluna JOB
dfori['JOB'].value_counts().plot.bar()

In [ ]:
# Exibindo gráfico de barras da coluna JOB com a coluna BAD
BADXJOB = pd.crosstab(dfori['JOB'],dfori['BAD'])
BADXJOB.div(BADXJOB.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='BAD X JOB', figsize=(8,8))

In [ ]:
# Analisando os dados da coluna REASON
dfori['REASON'].value_counts()

In [ ]:
# Exibindo gráfico de barras da coluna JOB
dfori['REASON'].value_counts().plot.bar()

In [ ]:
# Exibindo gráfico de barras da coluna REASON com a coluna BAD
BADXREASON = pd.crosstab(dfori['REASON'],dfori['BAD'])
BADXREASON.div(BADXREASON.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='BAD X REASON', figsize=(8,8))

In [ ]:
#Exibindo estatísticas básicas quando o empréstimo está em situação de inadimplência
dfori[dfori['BAD']==1].drop('BAD', axis=1).describe().style.format("{:.2f}")

In [ ]:
#Exibindo estatísticas básicas quando o empréstimo está em situação de normalidade
dfori[dfori['BAD']==0].drop('BAD', axis=1).describe().style.format("{:.2f}")

In [ ]:
# Preenchendo os valores nulos da base de dados
dfori = dfori.fillna({"VALUE": dfori['VALUE'].mean()//1, "MORTDUE": 0,  "DEROG": 0, "DELINQ": 0, "CLAGE": 0, 
                      "NINQ": 0, "CLNO": 0, "YOJ": dfori['YOJ'].mean()//1, "DEBTINC": dfori['DEBTINC'].mean()//1,
                      "REASON": 'Debtcon', "JOB": 'Other'})

In [ ]:
# Exibindo novamente as caracteristicas da base de dados
dfori.info()

In [ ]:
# Tranformando em categóricas as colunas do tipo "object"
for col in dfori.columns:
    if dfori[col].dtype == 'object':
        dfori[col]= dfori[col].astype('category').cat.codes

In [ ]:
# Exibindo novamente as caracteristicas da base de dados
dfori.info()

In [ ]:
#Exibindo estatísticas básicas
dfori.describe().style.format("{:.2f}")

In [ ]:
# Exibindo primeiros registros de forma transposta
dfori.head().T

In [ ]:
# Criando matriz de correlação entre as variáveis
dfori_matrix = dfori.corr()

In [ ]:
# Exibindo correlação das colunas com coluna BAD
dfori_matrix["BAD"].sort_values(ascending=False)

In [ ]:
# Criando uam nova coluna com a raz~~ao entre o valor do empréstimo e o valor do imóvel
dfori["LOAN_PER_VALUE"] = dfori["LOAN"]/dfori["VALUE"]

In [ ]:
# Recriando matriz de correlação entre as variáveis
dfori_matrix = dfori.corr()

In [ ]:
# Exibindo correlação das colunas com coluna BAD
dfori_matrix["BAD"].sort_values(ascending=False)

In [ ]:
# Criando uma cópia do dataframe
dfori2 = dfori.copy()

In [ ]:
# Selecionando as colunas para uso no modelo
feats = [c for c in dfori2.columns if c not in ['BAD']]

In [ ]:
# Exibindo as colunas
feats

In [ ]:
# Criando as bases de teste e treino
X, y = dfori2[feats], dfori2['BAD']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    random_state=0)
L_train, L_test, m_train, m_test = train_test_split(L, m, stratify=m,
                                                    random_state=0)

In [ ]:
# Criando as bases de teste e treino
L, m = dfori2[feats], dfori2['BAD']
L_train, L_test, m_train, m_test = train_test_split(L, m, stratify=m,
                                                    random_state=0)


In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200, learning_rate=0.4, random_state=42)
xgb.fit(L_train, m_train)
m_pred_xgb = xgb.predict(L_test)
print('XGB performance:')
print('Accuracy: {:.4f}'
      .format(accuracy_score(m_test, m_pred_xgb)))
print('Balanced accuracy: {:.4f}'
      .format(balanced_accuracy_score(m_test, m_pred_xgb)))
print('F1 Score: {:.4f}'
      .format(f1_score(m_test, m_pred_xgb)))


In [ ]:
# Mostrando as colunas mais importantes com com XGB
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Treinando os modelos para predição com o Random Forest com balanceamento com a apresentação das acurácias
brf = BalancedRandomForestClassifier(n_estimators=50, random_state=0,
                                     n_jobs=-1)
brf.fit(X_train, y_train)
y_pred_brf = brf.predict(X_test)
print('Balanced Random Forest classifier performance:')
print('Accuracy: {:.4f}'
      .format(accuracy_score(y_test, y_pred_brf)))
print('Balanced accuracy: {:.4f}'
      .format(balanced_accuracy_score(y_test, y_pred_brf)))
print('F1 Score: {:.4f}'
      .format(f1_score(y_test, y_pred_brf)))
            

In [ ]:
# Treinando os modelos para predição com o Random Forest som balanceamento com a apresentação das acurácias
rf = RandomForestClassifier(n_estimators=50, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Random Forest classifier performance:')
print('Accuracy: {:.4f}'
      .format(accuracy_score(y_test, y_pred_rf)))
print('Balanced accuracy: {:.4f}'
      .format(balanced_accuracy_score(y_test, y_pred_rf)))
print('F1 Score: {:.4f}'
      .format(f1_score(y_test, y_pred_rf)))


Conclusão

In [ ]:
Na aplicação dos modelos preditivos RandomForest (com e sem balanceamento) e XGBoost, o XGBoost apresentou melhor acurácia com os valores apurados, conforme abaixo:
    Accuracy: 0.9309
    Balanced accuracy: 0.8607
    F1 Score: 0.8110